In [82]:
import pandas as pd
import numpy as np

In [83]:
import warnings

# Suprimir todos los warnings
warnings.filterwarnings('ignore')

# Ahora puedes ejecutar tu código sin que aparezcan mensajes de advertencia

warnings.filterwarnings('ignore', category=UserWarning)

In [84]:
def analizar_dataframe(df):
    # Calcula la cantidad de nulos por columna
    nulos_por_columna = df.isnull().sum()

    # Calcula la cantidad de datos duplicados
    duplicados = df.duplicated().sum()

    # Calcula la cantidad de filas con algún outlier
    # Para este caso, vamos a considerar un valor como outlier si está fuera del rango de 1.5 veces el rango intercuartil (IQR)
    # Solo consideramos las columnas numéricas para calcular los cuantiles
    columnas_numericas = df.select_dtypes(include=[np.number]).columns
    Q1 = df[columnas_numericas].quantile(0.25)
    Q3 = df[columnas_numericas].quantile(0.75)
    IQR = Q3 - Q1
    outliers = ((df[columnas_numericas] < (Q1 - 1.5 * IQR)) | (df[columnas_numericas] > (Q3 + 1.5 * IQR))).sum()

    return nulos_por_columna, duplicados, outliers

In [85]:
!pip install openmeteo_requests
!pip install requests_cache
!pip install retry_requests

In [86]:
import openmeteo_requests

import requests_cache
import pandas as pd
from retry_requests import retry

# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = 3600)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://air-quality-api.open-meteo.com/v1/air-quality"
params = {
	"latitude": 40.7143,
	"longitude": -74.006,
	"current": ["european_aqi", "us_aqi", "pm10", "pm2_5", "carbon_monoxide", "nitrogen_dioxide", "sulphur_dioxide", "ozone", "uv_index"],
	"hourly": ["pm10", "pm2_5", "carbon_monoxide", "nitrogen_dioxide", "sulphur_dioxide", "ozone", "uv_index", "uv_index_clear_sky", "us_aqi", "us_aqi_pm2_5", "us_aqi_pm10", "us_aqi_nitrogen_dioxide", "us_aqi_carbon_monoxide", "us_aqi_ozone", "us_aqi_sulphur_dioxide"],
	"timezone": "America/New_York",
	"start_date": "2022-07-29",
	"end_date": "2023-11-23"
}
responses = openmeteo.weather_api(url, params=params)

# Process first location. Add a for-loop for multiple locations or weather models
response = responses[0]
print(f"Coordinates {response.Latitude()}°E {response.Longitude()}°N")
print(f"Elevation {response.Elevation()} m asl")
print(f"Timezone {response.Timezone()} {response.TimezoneAbbreviation()}")
print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")

# Current values. The order of variables needs to be the same as requested.
current = response.Current()
current_european_aqi = current.Variables(0).Value()
current_us_aqi = current.Variables(1).Value()
current_pm10 = current.Variables(2).Value()
current_pm2_5 = current.Variables(3).Value()
current_carbon_monoxide = current.Variables(4).Value()
current_nitrogen_dioxide = current.Variables(5).Value()
current_sulphur_dioxide = current.Variables(6).Value()
current_ozone = current.Variables(7).Value()
current_uv_index = current.Variables(8).Value()

print(f"Current time {current.Time()}")
print(f"Current european_aqi {current_european_aqi}")
print(f"Current us_aqi {current_us_aqi}")
print(f"Current pm10 {current_pm10}")
print(f"Current pm2_5 {current_pm2_5}")
print(f"Current carbon_monoxide {current_carbon_monoxide}")
print(f"Current nitrogen_dioxide {current_nitrogen_dioxide}")
print(f"Current sulphur_dioxide {current_sulphur_dioxide}")
print(f"Current ozone {current_ozone}")
print(f"Current uv_index {current_uv_index}")

# Process hourly data. The order of variables needs to be the same as requested.
hourly = response.Hourly()
hourly_pm10 = hourly.Variables(0).ValuesAsNumpy()
hourly_pm2_5 = hourly.Variables(1).ValuesAsNumpy()
hourly_carbon_monoxide = hourly.Variables(2).ValuesAsNumpy()
hourly_nitrogen_dioxide = hourly.Variables(3).ValuesAsNumpy()
hourly_sulphur_dioxide = hourly.Variables(4).ValuesAsNumpy()
hourly_ozone = hourly.Variables(5).ValuesAsNumpy()
hourly_uv_index = hourly.Variables(6).ValuesAsNumpy()
hourly_uv_index_clear_sky = hourly.Variables(7).ValuesAsNumpy()
hourly_us_aqi = hourly.Variables(8).ValuesAsNumpy()
hourly_us_aqi_pm2_5 = hourly.Variables(9).ValuesAsNumpy()
hourly_us_aqi_pm10 = hourly.Variables(10).ValuesAsNumpy()
hourly_us_aqi_nitrogen_dioxide = hourly.Variables(11).ValuesAsNumpy()
hourly_us_aqi_carbon_monoxide = hourly.Variables(12).ValuesAsNumpy()
hourly_us_aqi_ozone = hourly.Variables(13).ValuesAsNumpy()
hourly_us_aqi_sulphur_dioxide = hourly.Variables(14).ValuesAsNumpy()

hourly_data = {"date": pd.date_range(
	start = pd.to_datetime(hourly.Time(), unit = "s"),
	end = pd.to_datetime(hourly.TimeEnd(), unit = "s"),
	freq = pd.Timedelta(seconds = hourly.Interval()),
	inclusive = "left"
)}
hourly_data["pm10"] = hourly_pm10
hourly_data["pm2_5"] = hourly_pm2_5
hourly_data["carbon_monoxide"] = hourly_carbon_monoxide
hourly_data["nitrogen_dioxide"] = hourly_nitrogen_dioxide
hourly_data["sulphur_dioxide"] = hourly_sulphur_dioxide
hourly_data["ozone"] = hourly_ozone
hourly_data["uv_index"] = hourly_uv_index
hourly_data["uv_index_clear_sky"] = hourly_uv_index_clear_sky
hourly_data["us_aqi"] = hourly_us_aqi
hourly_data["us_aqi_pm2_5"] = hourly_us_aqi_pm2_5
hourly_data["us_aqi_pm10"] = hourly_us_aqi_pm10
hourly_data["us_aqi_nitrogen_dioxide"] = hourly_us_aqi_nitrogen_dioxide
hourly_data["us_aqi_carbon_monoxide"] = hourly_us_aqi_carbon_monoxide
hourly_data["us_aqi_ozone"] = hourly_us_aqi_ozone
hourly_data["us_aqi_sulphur_dioxide"] = hourly_us_aqi_sulphur_dioxide

hourly_dataframe = pd.DataFrame(data = hourly_data)
print(hourly_dataframe)


Coordinates 40.80000305175781°E -74.0°N
Elevation 51.0 m asl
Timezone b'America/New_York' b'EST'
Timezone difference to GMT+0 -18000 s
Current time 1701183600
Current european_aqi 24.0
Current us_aqi 24.96527862548828
Current pm10 4.900000095367432
Current pm2_5 3.4000000953674316
Current carbon_monoxide 228.0
Current nitrogen_dioxide 10.800000190734863
Current sulphur_dioxide 4.5
Current ozone 60.0
Current uv_index 1.149999976158142


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [87]:
hourly_dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11592 entries, 0 to 11591
Data columns (total 16 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   date                     11592 non-null  datetime64[ns]
 1   pm10                     11453 non-null  float32       
 2   pm2_5                    11453 non-null  float32       
 3   carbon_monoxide          11453 non-null  float32       
 4   nitrogen_dioxide         11453 non-null  float32       
 5   sulphur_dioxide          11453 non-null  float32       
 6   ozone                    11453 non-null  float32       
 7   uv_index                 11453 non-null  float32       
 8   uv_index_clear_sky       11453 non-null  float32       
 9   us_aqi                   11429 non-null  float32       
 10  us_aqi_pm2_5             11429 non-null  float32       
 11  us_aqi_pm10              11429 non-null  float32       
 12  us_aqi_nitrogen_dioxide  11451 n

In [88]:
hourly_dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11592 entries, 0 to 11591
Data columns (total 16 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   date                     11592 non-null  datetime64[ns]
 1   pm10                     11453 non-null  float32       
 2   pm2_5                    11453 non-null  float32       
 3   carbon_monoxide          11453 non-null  float32       
 4   nitrogen_dioxide         11453 non-null  float32       
 5   sulphur_dioxide          11453 non-null  float32       
 6   ozone                    11453 non-null  float32       
 7   uv_index                 11453 non-null  float32       
 8   uv_index_clear_sky       11453 non-null  float32       
 9   us_aqi                   11429 non-null  float32       
 10  us_aqi_pm2_5             11429 non-null  float32       
 11  us_aqi_pm10              11429 non-null  float32       
 12  us_aqi_nitrogen_dioxide  11451 n

In [89]:
hourly_dataframe.dropna(inplace = True)

In [90]:
hourly_dataframe.head(10)

,date,pm10,pm2_5,carbon_monoxide,nitrogen_dioxide,sulphur_dioxide,ozone,uv_index,uv_index_clear_sky,us_aqi,us_aqi_pm2_5,us_aqi_pm10,us_aqi_nitrogen_dioxide,us_aqi_carbon_monoxide,us_aqi_ozone,us_aqi_sulphur_dioxide
163,2022-08-05 00:00:00,39.500000,27.100000,295.0,63.099998,7.95,96.0,0.0,0.0,58.891846,58.891846,21.689392,31.077618,0.679147,23.519634,4.334787
164,2022-08-05 01:00:00,46.299999,31.900000,312.0,69.699997,7.90,83.0,0.0,0.0,59.459221,59.459221,22.034088,34.328209,0.716184,22.823902,4.307525
165,2022-08-05 02:00:00,41.599998,28.600000,336.0,79.000000,7.85,64.0,0.0,0.0,60.514187,60.514187,22.655300,38.908592,0.765298,21.567717,4.280262
166,2022-08-05 03:00:00,32.700001,22.600000,350.0,85.199997,7.55,46.0,0.0,0.0,61.524826,61.524826,23.231058,41.962173,0.827295,19.615801,4.116685
167,2022-08-05 04:00:00,29.799999,20.700001,333.0,80.449997,6.95,44.0,0.0,0.0,62.242908,62.242908,23.594696,39.622734,0.891707,17.200060,3.789531
168,2022-08-05 05:00:00,27.000000,18.799999,302.0,70.699997,6.20,47.0,0.0,0.0,62.916672,62.916672,23.920452,34.820724,0.942834,14.707019,3.380589
169,2022-08-05 06:00:00,24.500000,17.200001,262.0,56.000000,5.30,54.0,0.0,0.0,63.439716,63.439716,24.159088,27.580770,0.973027,12.407235,2.889858
170,2022-08-05 07:00:00,22.799999,15.900000,240.0,44.450001,4.90,60.0,0.0,0.0,63.758873,63.758873,24.284090,21.892239,0.982287,10.629251,2.671756
171,2022-08-05 08:00:00,21.600000,15.200000,220.0,31.799999,4.65,67.0,0.0,0.0,63.874115,63.874115,24.306816,15.661939,0.978261,9.547000,2.535442
172,2022-08-05 09:00:00,20.400000,14.300000,201.0,19.150000,4.45,74.0,0.0,0.0,63.794334,63.794334,24.227268,9.431639,0.948068,8.986549,2.426391


In [91]:
hourly_dataframe.tail(30)

,date,pm10,pm2_5,carbon_monoxide,nitrogen_dioxide,sulphur_dioxide,ozone,uv_index,uv_index_clear_sky,us_aqi,us_aqi_pm2_5,us_aqi_pm10,us_aqi_nitrogen_dioxide,us_aqi_carbon_monoxide,us_aqi_ozone,us_aqi_sulphur_dioxide
11562,2023-11-22 23:00:00,12.000000,8.4,343.0,51.700001,6.2,10.0,0.00,0.00,27.326389,27.326389,9.863636,25.462965,0.777375,7.053958,3.380589
11563,2023-11-23 00:00:00,16.100000,10.9,307.0,50.900002,5.7,10.0,0.00,0.00,27.465281,27.465281,9.821969,25.068954,0.806763,6.474181,3.107961
11564,2023-11-23 01:00:00,18.299999,12.6,318.0,51.799999,6.1,7.0,0.00,0.00,28.177080,28.177080,9.981061,25.512213,0.831321,5.701144,3.326064
11565,2023-11-23 02:00:00,18.200001,12.6,333.0,52.599998,6.6,3.0,0.00,0.00,29.253468,29.253468,10.246212,25.906225,0.874396,4.657545,3.598691
11566,2023-11-23 03:00:00,16.799999,11.6,337.0,49.799999,6.8,2.0,0.00,0.00,30.364582,30.364582,10.526516,24.527185,0.930757,3.459338,3.707743
11567,2023-11-23 04:00:00,13.800000,9.5,321.0,40.000000,6.4,7.0,0.00,0.00,31.267363,31.267363,10.734849,19.700552,0.983897,2.396413,3.489640
11568,2023-11-23 05:00:00,10.200000,7.1,294.0,26.600000,5.6,15.0,0.00,0.00,31.770834,31.770834,10.814392,13.100867,1.018518,1.700680,3.053435
11569,2023-11-23 06:00:00,7.300000,5.1,271.0,16.200001,4.9,23.0,0.00,0.00,31.840282,31.840282,10.750000,7.978724,1.029388,1.410792,2.671756
11570,2023-11-23 07:00:00,4.400000,3.0,256.0,11.200000,4.2,30.0,0.00,0.00,31.614586,31.614586,10.590909,5.516154,1.016103,1.488095,2.290076
11571,2023-11-23 08:00:00,2.700000,1.9,244.0,9.300000,3.6,36.0,0.00,0.00,30.937504,30.937504,10.287879,4.580379,0.981079,1.874613,1.962923


In [92]:
hourly_dataframe.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11427 entries, 163 to 11591
Data columns (total 16 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   date                     11427 non-null  datetime64[ns]
 1   pm10                     11427 non-null  float32       
 2   pm2_5                    11427 non-null  float32       
 3   carbon_monoxide          11427 non-null  float32       
 4   nitrogen_dioxide         11427 non-null  float32       
 5   sulphur_dioxide          11427 non-null  float32       
 6   ozone                    11427 non-null  float32       
 7   uv_index                 11427 non-null  float32       
 8   uv_index_clear_sky       11427 non-null  float32       
 9   us_aqi                   11427 non-null  float32       
 10  us_aqi_pm2_5             11427 non-null  float32       
 11  us_aqi_pm10              11427 non-null  float32       
 12  us_aqi_nitrogen_dioxide  11427

pm10 (Partículas en Suspensión de Menos de 10 Micrómetros):

Son partículas finas en el aire con un diámetro menor o igual a 10 micrómetros.
Pueden provenir de diversas fuentes, como la combustión de motores, la industria y el polvo del suelo.
Altas concentraciones pueden afectar la salud respiratoria.


pm2_5 (Partículas en Suspensión de Menos de 2.5 Micrómetros):

Son partículas más pequeñas y finas que las PM10.
También pueden provenir de diversas fuentes, incluyendo la quema de combustibles fósiles.
Debido a su tamaño, pueden penetrar más profundamente en los pulmones y están asociadas con problemas respiratorios y cardiovasculares.


carbon_monoxide (Monóxido de Carbono):

Es un gas incoloro e inodoro producido por la combustión incompleta de combustibles.
Altas concentraciones pueden afectar la capacidad del cuerpo para transportar oxígeno, causando problemas de salud.
nitrogen_dioxide (Dióxido de Nitrógeno):

Es un gas producido por la combustión de combustibles fósiles.
Puede irritar los pulmones y contribuir a la formación de partículas finas y ozono troposférico.


sulphur_dioxide (Dióxido de Azufre):

Es un gas incoloro con olor a huevo podrido.
Proviene principalmente de la combustión de carbón y petróleo.
Puede irritar los ojos y las vías respiratorias y contribuir a la formación de partículas finas.


ozone (Ozono):

Es un gas incoloro y altamente reactivo.
En la troposfera, es un contaminante del aire secundario formado por reacciones químicas entre óxidos de nitrógeno y compuestos orgánicos volátiles.
Altas concentraciones pueden causar problemas respiratorios.


aerosol_optical_depth (Profundidad Óptica del Aerosol):

Mide la absorción y dispersión de la radiación solar por partículas en suspensión en la atmósfera.
Puede influir en la visibilidad y tiene implicaciones climáticas.


dust (Polvo Atmosférico):

Se refiere a partículas sólidas en suspensión en el aire, como polvo y arena.
Puede afectar la calidad del aire y tener impactos en la salud respiratoria.


uv_index (Índice UV):

Mide la intensidad de la radiación ultravioleta del sol.
Un índice UV alto indica una mayor exposición a la radiación UV, lo que puede aumentar el riesgo de daño a la piel y los ojos.


uv_index_clear_sky (Índice UV en Cielo Despejado):

Similar al índice UV, pero se refiere a la intensidad de la radiación ultravioleta en un cielo despejado sin nubes.
Estas variables son medidas comunes utilizadas para evaluar la calidad del aire y monitorear sus posibles impactos en la salud humana y el medio ambiente.



Veamos las nuevas variables que has agregado a tu DataFrame:

us_aqi (Índice de Calidad del Aire en EE. UU.):

Es un índice que resume la calidad del aire en función de varios contaminantes, incluyendo PM10, PM2.5, dióxido de nitrógeno, monóxido de carbono, ozono y dióxido de azufre.
Se utiliza para proporcionar una visión general de la calidad del aire y se clasifica en diferentes niveles, como bueno, moderado, insalubre, etc.


us_aqi_pm2_5 (Índice de Calidad del Aire en EE. UU. para PM2.5):

Especifica el índice de calidad del aire específico para partículas en suspensión PM2.5.


us_aqi_pm10 (Índice de Calidad del Aire en EE. UU. para PM10):

Especifica el índice de calidad del aire específico para partículas en suspensión PM10.


us_aqi_nitrogen_dioxide (Índice de Calidad del Aire en EE. UU. para Dióxido de Nitrógeno):

Especifica el índice de calidad del aire específico para el dióxido de nitrógeno.


us_aqi_carbon_monoxide (Índice de Calidad del Aire en EE. UU. para Monóxido de Carbono):

Especifica el índice de calidad del aire específico para el monóxido de carbono.


us_aqi_ozone (Índice de Calidad del Aire en EE. UU. para Ozono):

Especifica el índice de calidad del aire específico para el ozono.


us_aqi_sulphur_dioxide (Índice de Calidad del Aire en EE. UU. para Dióxido de Azufre):

Especifica el índice de calidad del aire específico para el dióxido de azufre.
Estas nuevas variables, conocidas como índices de calidad del aire, proporcionan una métrica compuesta que puede ayudar a comprender la calidad general del aire. Los valores más altos indican una peor calidad del aire, mientras que los valores más bajos son indicativos de una mejor calidad del aire. Estos índices son útiles para evaluar rápidamente la calidad del aire y tomar decisiones basadas en ese contexto.

In [93]:
nulos_hourly, duplicados_hourly, outliers_hourly = analizar_dataframe(hourly_dataframe)

In [94]:
nulos_hourly, duplicados_hourly, outliers_hourly = analizar_dataframe(hourly_dataframe)

# Imprime los resultados
print("HOurly Air Quality NY :")
print()
print("Nulos por columna:", nulos_hourly)
print()
print("Duplicados:", duplicados_hourly)
print()
print("Outliers:", outliers_hourly)
print()

HOurly Air Quality NY :

Nulos por columna: date                       0
pm10                       0
pm2_5                      0
carbon_monoxide            0
nitrogen_dioxide           0
sulphur_dioxide            0
ozone                      0
uv_index                   0
uv_index_clear_sky         0
us_aqi                     0
us_aqi_pm2_5               0
us_aqi_pm10                0
us_aqi_nitrogen_dioxide    0
us_aqi_carbon_monoxide     0
us_aqi_ozone               0
us_aqi_sulphur_dioxide     0
dtype: int64

Duplicados: 0

Outliers: pm10                        535
pm2_5                       569
carbon_monoxide             874
nitrogen_dioxide            164
sulphur_dioxide             373
ozone                       299
uv_index                   1334
uv_index_clear_sky         1228
us_aqi                      210
us_aqi_pm2_5                206
us_aqi_pm10                 289
us_aqi_nitrogen_dioxide     164
us_aqi_carbon_monoxide      847
us_aqi_ozone                327
us_aq

In [95]:
hourly_dataframe.dropna(inplace = True)

In [96]:
hourly_dataframe.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11427 entries, 163 to 11591
Data columns (total 16 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   date                     11427 non-null  datetime64[ns]
 1   pm10                     11427 non-null  float32       
 2   pm2_5                    11427 non-null  float32       
 3   carbon_monoxide          11427 non-null  float32       
 4   nitrogen_dioxide         11427 non-null  float32       
 5   sulphur_dioxide          11427 non-null  float32       
 6   ozone                    11427 non-null  float32       
 7   uv_index                 11427 non-null  float32       
 8   uv_index_clear_sky       11427 non-null  float32       
 9   us_aqi                   11427 non-null  float32       
 10  us_aqi_pm2_5             11427 non-null  float32       
 11  us_aqi_pm10              11427 non-null  float32       
 12  us_aqi_nitrogen_dioxide  11427

In [97]:
hourly_dataframe.head()

,date,pm10,pm2_5,carbon_monoxide,nitrogen_dioxide,sulphur_dioxide,ozone,uv_index,uv_index_clear_sky,us_aqi,us_aqi_pm2_5,us_aqi_pm10,us_aqi_nitrogen_dioxide,us_aqi_carbon_monoxide,us_aqi_ozone,us_aqi_sulphur_dioxide
163,2022-08-05 00:00:00,39.500000,27.100000,295.0,63.099998,7.95,96.0,0.0,0.0,58.891846,58.891846,21.689392,31.077618,0.679147,23.519634,4.334787
164,2022-08-05 01:00:00,46.299999,31.900000,312.0,69.699997,7.90,83.0,0.0,0.0,59.459221,59.459221,22.034088,34.328209,0.716184,22.823902,4.307525
165,2022-08-05 02:00:00,41.599998,28.600000,336.0,79.000000,7.85,64.0,0.0,0.0,60.514187,60.514187,22.655300,38.908592,0.765298,21.567717,4.280262
166,2022-08-05 03:00:00,32.700001,22.600000,350.0,85.199997,7.55,46.0,0.0,0.0,61.524826,61.524826,23.231058,41.962173,0.827295,19.615801,4.116685
167,2022-08-05 04:00:00,29.799999,20.700001,333.0,80.449997,6.95,44.0,0.0,0.0,62.242908,62.242908,23.594696,39.622734,0.891707,17.200060,3.789531


Los niveles considerados saludables para cada uno de los parámetros del Índice de Calidad del Aire (AQI) varían según las pautas establecidas por las agencias de control de la calidad del aire. Aquí hay una guía general basada en las categorías comúnmente utilizadas en el AQI:

AQI General:

0-50 (Bueno): La calidad del aire se considera satisfactoria y la contaminación del aire presenta un bajo riesgo o ninguno.

51-100 (Moderado): La calidad del aire es aceptable; sin embargo, para algunos contaminantes puede haber una moderada preocupación para un número muy pequeño de personas que son inusualmente sensibles a la contaminación del aire.

101-150 (No Saludable para Grupos Sensibles): Los miembros de grupos sensibles pueden experimentar efectos en la salud. El público en general no es probable que se vea afectado.

151-200 (No Saludable): Todos pueden comenzar a experimentar efectos en la salud; los miembros de grupos sensibles pueden experimentar efectos más graves.

201-300 (Muy No Saludable): Advertencia de salud: todos pueden comenzar a experimentar efectos de salud más graves.

301 y superior (Peligroso): Advertencia de salud de emergencia: toda la población es más probable que se vea afectada.

Para parámetros específicos:

PM2.5 y PM10:

0-12: Bueno
13-35: Moderado
36-55: Poco saludable para grupos sensibles
56-150: Poco saludable
151-250: Muy poco saludable
251 y superior: Peligroso


NO2:

0-53: Bueno
54-100: Moderado
101-360: Poco saludable para grupos sensibles
361-650: Poco saludable
651-1250: Muy poco saludable
1251 y superior: Peligroso


CO:

0-4.4: Bueno
4.5-9.4: Moderado
9.5-12.4: Poco saludable para grupos sensibles
12.5-15.4: Poco saludable
15.5-30.4: Muy poco saludable
30.5 y superior: Peligroso


O3:

0-70: Bueno
71-95: Moderado
96-125: Poco saludable para grupos sensibles
126-165: Poco saludable
166-205: Muy poco saludable
206 y superior: Peligroso


SO2:

0-35: Bueno
36-75: Moderado
76-185: Poco saludable para grupos sensibles
186-304: Poco saludable
305-604: Muy poco saludable
605 y superior: Peligroso

In [98]:

from datetime import datetime, timedelta

# Asegurarse de que la columna 'date' esté en formato datetime
hourly_dataframe['date'] = pd.to_datetime(hourly_dataframe['date'])



In [99]:
hourly_dataframe.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11427 entries, 163 to 11591
Data columns (total 16 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   date                     11427 non-null  datetime64[ns]
 1   pm10                     11427 non-null  float32       
 2   pm2_5                    11427 non-null  float32       
 3   carbon_monoxide          11427 non-null  float32       
 4   nitrogen_dioxide         11427 non-null  float32       
 5   sulphur_dioxide          11427 non-null  float32       
 6   ozone                    11427 non-null  float32       
 7   uv_index                 11427 non-null  float32       
 8   uv_index_clear_sky       11427 non-null  float32       
 9   us_aqi                   11427 non-null  float32       
 10  us_aqi_pm2_5             11427 non-null  float32       
 11  us_aqi_pm10              11427 non-null  float32       
 12  us_aqi_nitrogen_dioxide  11427

Eliminacion de datos irrisorios

In [100]:
import pandas as pd

# Supongamos que tu DataFrame se llama hourly_dataframe
# Lista de columnas numéricas en las que deseas aplicar el filtro
numeric_columns = ['pm10', 'pm2_5', 'carbon_monoxide', 'nitrogen_dioxide', 'sulphur_dioxide', 'ozone', 'uv_index', 'uv_index_clear_sky', 'us_aqi', 'us_aqi_pm2_5', 'us_aqi_pm10', 'us_aqi_nitrogen_dioxide', 'us_aqi_carbon_monoxide', 'us_aqi_ozone', 'us_aqi_sulphur_dioxide']

# Aplicar el filtro para valores negativos y extremadamente altos
for col in numeric_columns:
    hourly_dataframe = hourly_dataframe[(hourly_dataframe[col] >= 0) & (hourly_dataframe[col] <= 750)]

# Esto eliminará las filas que contienen valores negativos o extremadamente altos en las columnas numéricas.



In [101]:
hourly_dataframe.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11032 entries, 163 to 11591
Data columns (total 16 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   date                     11032 non-null  datetime64[ns]
 1   pm10                     11032 non-null  float32       
 2   pm2_5                    11032 non-null  float32       
 3   carbon_monoxide          11032 non-null  float32       
 4   nitrogen_dioxide         11032 non-null  float32       
 5   sulphur_dioxide          11032 non-null  float32       
 6   ozone                    11032 non-null  float32       
 7   uv_index                 11032 non-null  float32       
 8   uv_index_clear_sky       11032 non-null  float32       
 9   us_aqi                   11032 non-null  float32       
 10  us_aqi_pm2_5             11032 non-null  float32       
 11  us_aqi_pm10              11032 non-null  float32       
 12  us_aqi_nitrogen_dioxide  11032

In [102]:
nulos_hourly, duplicados_hourly, outliers_hourly = analizar_dataframe(hourly_dataframe)

# Imprime los resultados
print("HOurly Air Quality NY :")
print()
print("Nulos por columna:", nulos_hourly)
print()
print("Duplicados:", duplicados_hourly)
print()
print("Outliers:", outliers_hourly)
print()

HOurly Air Quality NY :

Nulos por columna: date                       0
pm10                       0
pm2_5                      0
carbon_monoxide            0
nitrogen_dioxide           0
sulphur_dioxide            0
ozone                      0
uv_index                   0
uv_index_clear_sky         0
us_aqi                     0
us_aqi_pm2_5               0
us_aqi_pm10                0
us_aqi_nitrogen_dioxide    0
us_aqi_carbon_monoxide     0
us_aqi_ozone               0
us_aqi_sulphur_dioxide     0
dtype: int64

Duplicados: 0

Outliers: pm10                        378
pm2_5                       413
carbon_monoxide             622
nitrogen_dioxide            151
sulphur_dioxide             265
ozone                       314
uv_index                   1237
uv_index_clear_sky         1135
us_aqi                      187
us_aqi_pm2_5                182
us_aqi_pm10                 260
us_aqi_nitrogen_dioxide     153
us_aqi_carbon_monoxide      724
us_aqi_ozone                316
us_aq

In [103]:
hourly_dataframe.to_parquet('air_quality_clean.parquet')

In [104]:
hourly_dataframe.to_csv('air_quality_clean.csv')

In [105]:
len(hourly_dataframe)

11032